In [ ]:
from pyspark.sql import SparkSession
from  pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.window import Window


In [10]:
spark =(
    SparkSession
        .builder
        .config('spark.driver.memory','10g')
        .master('local[*]')
        .appName('bucketing')
        .getOrCreate()
)
spark.sparkContext.setLogLevel("ERROR")
spark.conf.set('spark.sql.autoBroadcastJoinThreshold',-1)

In [11]:
spark

In [ ]:
orders_df = spark.read.csv('data/orders.csv',inferSchema=True,header=True)

In [15]:
orders_df.printSchema()
orders_df.show(5)

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- total_amount: integer (nullable = true)

+--------+----------+-----------+--------+----------+------------+
|order_id|product_id|customer_id|quantity|order_date|total_amount|
+--------+----------+-----------+--------+----------+------------+
|       1|        80|         10|       4| 2023-3-20|        1003|
|       2|        69|         30|       3|2023-12-11|         780|
|       3|        61|         20|       4| 2023-4-26|        1218|
|       4|        62|         44|       3| 2023-8-26|        2022|
|       5|        78|         46|       4|  2023-8-5|        1291|
+--------+----------+-----------+--------+----------+------------+
only showing top 5 rows



In [16]:
products_df = spark.read.csv('data/products.csv',inferSchema=True,header=True)

In [17]:
products_df.printSchema()
products_df.show(5)

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- stock: integer (nullable = true)

+----------+------------+-----------+-------+-----+-----+
|product_id|product_name|   category|  brand|price|stock|
+----------+------------+-----------+-------+-----+-----+
|         1|   Product_1|Electronics|Brand_4|   26|  505|
|         2|   Product_2|    Apparel|Brand_4|  489|   15|
|         3|   Product_3|    Apparel|Brand_4|  102|  370|
|         4|   Product_4|  Groceries|Brand_1|   47|  433|
|         5|   Product_5|  Groceries|Brand_3|  244|  902|
+----------+------------+-----------+-------+-----+-----+
only showing top 5 rows



In [19]:
(
    orders_df.write.mode('overwrite').saveAsTable('orders_bucket')
)

In [20]:
(
    products_df.write.mode('overwrite').saveAsTable('products_bucket')
)

In [23]:
spark.sql('select * from orders_bucket').explain()

== Physical Plan ==
*(1) ColumnarToRow
+- FileScan parquet default.orders_bucket[order_id#142,product_id#143,customer_id#144,quantity#145,order_date#146,total_amount#147] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex[file:/home/hari/python-notebooks/spark-warehouse/orders_bucket], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<order_id:int,product_id:int,customer_id:int,quantity:int,order_date:string,total_amount:int>


